In [ ]:
import time
import datetime
import numpy as np
import soundfile
import uwacan

In [ ]:
import abc
class Parent(abc.ABC):
    @staticmethod
    def makeprop(name):
        def getter(self):
            return getattr(self, '_' + name)
        return property(getter)

    @property
    @abc.abstractmethod
    def x(self):
        ...

class Child(Parent):
    def __init__(self, x):
        self._x = x
    x = Parent.makeprop('x')

class Second(Parent):
    def __init__(self, x):
        self._x = x

c = Child(5)
print(c.x)
d = Second(5)

In [ ]:
class File:
    def __init__(self, proper):
        self.proper = proper
        print('Initializing File')
        return self.__bool__()

    def __bool__(self):
        print('checking proper')
        return self.proper

class Container:
    class File(File):
        def __init__(self, proper, nice):
            print('initializing Container.File')
            if not super().__init__(proper):
                print('super init incomplete')
                return
            self.nice = nice

        def __bool__(self):
            if not super().__bool__():
                return False
            print('check nice')
            return self.nice

    def __init__(self, proper, nice):
        print('Initializing Container')
        if file := self.File(proper, nice):
            self.file = file
            print('Is proper and nice')

c = Container(False, True)

In [ ]:
os.walk?

In [ ]:
import re
import os

In [ ]:
def check_w_regex(path):
    for date in os.listdir(path):
        if not re.match(r'\d{4}-\d{2}-\d{2}', date):
            continue
        for file in os.listdir(os.path.join(path, date)):
            ...

def check_w_isdir(path):
    for date in os.listdir(path):
        if not os.path.isdir(os.path.join(path, date)):
            continue
        for file in os.listdir(os.path.join(path, date)):
            ...

def with_walk(path):
    for root, dirs, files in os.walk(path):
        for file in files:
            ...

def with_scandir(path):
    for date in sorted(filter(lambda d: d.is_dir(), os.scandir(path)), key=lambda d: d.name):
        print(date.name)
        if date.is_dir():
            print('is a dir')
            for file in os.scandir(date.path):
                ...

check_w_regex(path)
check_w_isdir(path)
with_walk(path)
with_scandir(path)

In [ ]:
path = r'C:\Users\carl4189\OneDrive - IVL Svenska Miljöinstitutet AB\Kalibrering\RTSYS_data\Enhet A'
dir = next(os.scandir(path))
list(dir)

In [11]:
path = 'D:\\2022_10_Mälaren\\RTsys1\\'
# path = r'C:\Users\carl4189\OneDrive - IVL Svenska Miljöinstitutet AB\Kalibrering\RTSYS_data\Enhet A'

In [12]:
rtsys = uwacan.recordings.SylenceLP(path)

In [ ]:
for file in rtsys.files:
    print(file.start_time, hasattr(file, '_stop_time'))

In [ ]:
file = rtsys.files[0]
print(file.name)
print(file.start_time)
file.read_info()
print(file.start_time)

In [ ]:
time_deltas = []
start_in_name = []
for file in rtsys.files:
    from_name = file.start_time
    start_in_name.append(from_name)
    file.read_info()
    from_header = file.start_time
    time_deltas.append((from_header - from_name).total_seconds())
time_deltas = np.asarray(time_deltas)

In [ ]:
import plotly.graph_objects as go
go.Figure(go.Scatter(x=start_in_name, y=time_deltas))

In [ ]:
file = rtsys.files[-2]
file.start_time, file.bitdepth, file.samplerate, file.gain

In [14]:
segment = rtsys[uwacan.positional.TimeWindow(center="2022-10-5 06:08 +2", duration=120)]
data = segment.data
print(data.data.shape)

(15360000,)


In [16]:
data.data.size / 120

128000.0

In [15]:
10*np.log10(np.mean(data.data**2) / 1e-12)


126.40008183214748

In [ ]:
20 * np.log10(5.62341)

In [ ]:
rtsys.calibration

In [ ]:
rtsys.files[0].hydrophone_sensitivity

In [ ]:
file_value = 1
hydrophone_sens = -178
gain = 15
voltage_range = 2.5

calibration = hydrophone_sens + gain - 20*np.log10(voltage_range)
c = 10**(calibration / 20) / 1e-6
pressure = file_value / c
print(20*np.log10(pressure / 1e-6))

In [ ]:
data.data.size / rtsys.samplerate

In [ ]:
import re
contents = open(r'D:\2022_10_Mälaren\RTsys1\2022-09-20\log_2022-09-20_10-00-03.txt', 'r').read()
print(len(contents))
# contents = re.sub(r'\[.*?m', '', contents)
# contents = re.sub(r"^\[0;33m\[[ \d]{8}\] \[msd:\d files:\d \(e:\d\) max:[ \d]{6}\/[ \d]{6}\/[ \d]{6}\/[ \d]{6} size written:[ \d]{3}\/[ \d]{3}MB adc:[ \d]{6}\(p:\d\)\n", '', contents, flags=re.MULTILINE)
# contents = re.sub(r"^\[[ \d]+?\] sd:\d files:\d+? \(e:\d\) max:[ \d]+?\/[ \d]+?\/[ \d]+?\/[ \d]+? size written:[ \d]+?\/[ \d]+?MB adc:[ \d]+?\(p:\d\)\n", '', contents, flags=re.MULTILINE)
# contents = re.sub(r"^\[[ \d]+?\] Vin = \d+\.\dV, Iin = \d+\.\d+A, Power = \d+\.\d+W, Temp = \d+\.\d+Â°C\n", '', contents, flags=re.MULTILINE)
# contents = re.sub(r"^\[[ \d]+?\] status RTC: 0xC1, serial: 0x\d+?, logs: 0x00, int. ADC: 0x00, ÂµP: 0\n", '', contents, flags=re.MULTILINE)
# contents = re.sub(r"^\[[ \d]+?\] PSU input OK: 1, PSU output OK: 1\n", '', contents, flags=re.MULTILINE)

contents = re.sub(
    r"\[.*?m\[[ \d]+?\] \[.*?m("
    r"sd:\d files:\d+? \(e:\d\) max:[ \d]+?\/[ \d]+?\/[ \d]+?\/[ \d]+? size written:[ \d]+?\/[ \d]+?MB adc:[ \d]+?\(p:\d\)"
    r"|Vin = \d+\.\dV, Iin = \d+\.\d+A, Power = \d+\.\d+W, Temp = \d+\.\d+Â°C"
    r"|status RTC: 0xC1, serial: 0x\d+?, logs: 0x00, int. ADC: 0x00, ÂµP: 0"
    r"|PSU input OK: 1, PSU output OK: 1"
    r")\n"
    , '', contents
)

# contents = re.sub(
#     r"^\[.*?m\[[ \d]+?\] \[.*?m"
#     r"sd:\d files:\d+? \(e:\d\) max:[ \d]+?\/[ \d]+?\/[ \d]+?\/[ \d]+? size written:[ \d]+?\/[ \d]+?MB adc:[ \d]+?\(p:\d\)"
#     # r"|Vin = \d+\.\dV, Iin = \d+\.\d+A, Power = \d+\.\d+W, Temp = \d+\.\d+Â°C"
#     # r"|status RTC: 0xC1, serial: 0x\d+?, logs: 0x00, int. ADC: 0x00, ÂµP: 0"
#     # r"|PSU input OK: 1, PSU output OK: 1"
#     "\n", '', contents, re.MULTILINE)
# con
# contents = re.sub(r'^\[0;33m\[[ \d]{8}\] \[msd:\d files:\d \(e:\d\) max:[ \d]{6}\/[ \d]{6}\/[ \d]{6}\/[ \d]{6} size written:[ \d]{3}\/[ \d]{3}MB adc:[ \d]{6}\(p:\d\)\n', '', contents, flags=re.MULTILINE)
for line in contents.split('\n')[100:109]:
    print(line)
print(len(contents))
# with open('temp log.txt', 'w') as file:
    # file.write(contents)

In [ ]:
line

In [ ]:
print(
    "foo"
    "bar"
)

In [ ]:
file = rtsys.files[489]
print(file.name)
print(file.start_time)
print(file.stop_time)

In [ ]:
file_created_at = datetime.datetime(2022,9,20,10,0,0) + datetime.timedelta(seconds=1280828526e-3)
file_records_at = datetime.datetime(2022,9,20,10,0,0) + datetime.timedelta(seconds=1282139501e-3)
print(file_created_at)
print(file_records_at)

In [ ]:
contents = open(r'D:\2022_10_Mälaren\RTsys1\2022-09-20\log_2022-09-20_10-00-03.txt', 'r').read()
matches = re.findall(r'^\[0;33m\[[ \d]{8}\] \[msd:\d files:\d \(e:\d\) max:[ \d]{6}\/[ \d]{6}\/[ \d]{6}\/[ \d]{6} size written:[ \d]{3}\/[ \d]{3}MB adc:[ \d]{6}\(p:\d\)\n', contents, flags=re.MULTILINE)
len(matches)

In [ ]:
print(contents)

In [ ]:
with open(r'D:\2022_10_Mälaren\RTsys1\2022-09-20\log_2022-09-20_10-00-03.txt', 'r') as input_file, open(r'temp_log.txt', 'w') as output_file:
    for line in input_file:
        if 'sd:' in line and 'files:' in line and 